----
# I. Data preprocessing

## 1. Read dataset

In [53]:
import pandas as pd

train = pd.read_csv("../dataset/train.csv")
test = pd.read_csv("../dataset/test.csv")
submission = pd.read_csv("../dataset/sample_submission.csv")

## 2. Split `train` to `sub_train` and `sub_test`

In [54]:
sub_test = pd.DataFrame()

for store in range(1, train["Store"].max() + 1):
    splited = train[train["Store"] == store].tail(4)
    train = train.drop(splited.index.to_list())
    sub_test = pd.concat([sub_test, splited])

sub_test = sub_test.reset_index(drop=1)
sub_train = train.reset_index(drop=1)

In [55]:
sub_train

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,1,1,05/02/2010,42.31,2.572,NaN,NaN,NaN,NaN,NaN,8.106,False,1643690.90
1,2,1,12/02/2010,38.51,2.548,NaN,NaN,NaN,NaN,NaN,8.106,True,1641957.44
2,3,1,19/02/2010,39.93,2.514,NaN,NaN,NaN,NaN,NaN,8.106,False,1611968.17
3,4,1,26/02/2010,46.63,2.561,NaN,NaN,NaN,NaN,NaN,8.106,False,1409727.59
4,5,1,05/03/2010,46.50,2.625,NaN,NaN,NaN,NaN,NaN,8.106,False,1554806.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,6247,45,03/08/2012,76.58,3.654,24853.05,39.56,17.96,11142.69,2768.32,8.684,False,725729.51
6071,6248,45,10/08/2012,78.65,3.722,17868.84,50.60,57.66,2593.93,1890.59,8.684,False,733037.32
6072,6249,45,17/08/2012,75.71,3.807,3657.79,6.00,0.30,1630.50,3794.22,8.684,False,722496.93
6073,6250,45,24/08/2012,72.62,3.834,7936.20,58.38,22.00,5518.07,2291.97,8.684,False,718232.26


In [56]:
sub_test

,id,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales
0,136,1,07/09/2012,83.96,3.730,5204.68,35.74,50.94,4120.32,2737.17,6.908,True,1661767.33
1,137,1,14/09/2012,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,6.908,False,1517428.87
2,138,1,21/09/2012,69.87,3.721,6352.30,7.64,4.69,1010.06,6456.71,6.908,False,1506126.06
3,139,1,28/09/2012,76.08,3.666,3666.27,7.64,1.65,1417.96,4744.28,6.908,False,1437059.26
4,275,2,07/09/2012,87.65,3.730,10658.86,149.28,60.97,2920.90,3275.57,6.565,True,1898777.07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,6116,44,28/09/2012,64.80,3.821,380.96,NaN,NaN,NaN,2022.03,5.407,False,355307.94
176,6252,45,07/09/2012,75.70,3.911,11024.45,12.80,52.63,1854.77,2055.70,8.684,True,766512.66
177,6253,45,14/09/2012,67.87,3.948,11407.95,NaN,4.30,3421.72,5268.92,8.684,False,702238.27
178,6254,45,21/09/2012,65.32,4.038,8452.20,92.28,63.24,2376.38,8670.40,8.684,False,723086.20


## 2. Missing values handling

### a) `sub_train`

In [57]:
sub_train = sub_train.fillna(0)

### b) `sub_test` and `test`

In [58]:
def each_means(df):
    means = {}
    for i in range(1, 6):
        means[f"Promotion{i}"] = df[f"Promotion{i}"].mean()
    return means

sub_test = sub_test.fillna(value=each_means(sub_test))
test = test.fillna(value=each_means(test))

## 3. Data manipulation

### a) `Date`

In [59]:
import datetime as dt

def date_to_week(date):
    day, month, year = map(int, date.split('/'))
    t = dt.datetime(year, month, day) - dt.datetime(2010, 2, 5)
    return t.days // 7


def date_split(date):
    return tuple(map(int, date.split('/')))


sub_train["Day"] = sub_train["Date"].apply(lambda x: date_split(x)[0])
sub_train["Month"] = sub_train["Date"].apply(lambda x: date_split(x)[1])
sub_train["Year"] = sub_train["Date"].apply(lambda x: date_split(x)[2])

sub_test["Day"] = sub_test["Date"].apply(lambda x: date_split(x)[0])
sub_test["Month"] = sub_test["Date"].apply(lambda x: date_split(x)[1])
sub_test["Year"] = sub_test["Date"].apply(lambda x: date_split(x)[2])

test["Day"] = test["Date"].apply(lambda x: date_split(x)[0])
test["Month"] = test["Date"].apply(lambda x: date_split(x)[1])
test["Year"] = test["Date"].apply(lambda x: date_split(x)[2])

### b) `IsHoliday`

In [60]:
sub_train["IsHoliday"] = sub_train["IsHoliday"].apply(int)
sub_test["IsHoliday"] = sub_test["IsHoliday"].apply(int)
test["IsHoliday"] = test["IsHoliday"].apply(int)

### c) `Store`

In [61]:
sub_train = pd.get_dummies(data=sub_train, columns=["Store"])
sub_test = pd.get_dummies(data=sub_test, columns=["Store"])
test = pd.get_dummies(data=test, columns=["Store"])

### d) `Promotion1`, ... , `Promotion5`

In [62]:
# Scaling
from sklearn.preprocessing import RobustScaler


promos = ['Promotion1','Promotion2','Promotion3','Promotion4','Promotion5']

scaler = RobustScaler()
scaler.fit(sub_train[promos])
sub_train[promos] = scaler.transform(sub_train[promos])
sub_test[promos] = scaler.transform(sub_test[promos])
test[promos] = scaler.transform(test[promos])

### e) `Weekly_Sales` 

In [63]:
import numpy as np

sub_train["Weekly_Sales"] = np.log1p(sub_train["Weekly_Sales"])
sub_test["Weekly_Sales"] = np.log1p(sub_test["Weekly_Sales"])

## 4. Remove not using features

In [64]:
remove = ['id','Date']

sub_train = sub_train.drop(columns=remove)
sub_test = sub_test.drop(columns=remove)
test = test.drop(columns=remove)

In [65]:
sub_train

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,42.31,2.572,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.312455,...,0,0,0,0,0,0,0,0,0,0
1,38.51,2.548,0.000000,0.00,0.000000,0.000000,0.000000,8.106,1,14.311400,...,0,0,0,0,0,0,0,0,0,0
2,39.93,2.514,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.292967,...,0,0,0,0,0,0,0,0,0,0
3,46.63,2.561,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.158908,...,0,0,0,0,0,0,0,0,0,0
4,46.50,2.625,0.000000,0.00,0.000000,0.000000,0.000000,8.106,0,14.256862,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6070,76.58,3.654,23.066225,39.56,9.047859,210.577152,1.786779,8.684,0,13.494934,...,0,0,0,0,0,0,0,0,0,1
6071,78.65,3.722,16.584149,50.60,29.047859,49.020694,1.220259,8.684,0,13.504953,...,0,0,0,0,0,0,0,0,0,1
6072,75.71,3.807,3.394811,6.00,0.151134,30.813569,2.448935,8.684,0,13.490470,...,0,0,0,0,0,0,0,0,0,1
6073,72.62,3.834,7.365622,58.38,11.083123,104.281773,1.479325,8.684,0,13.484550,...,0,0,0,0,0,0,0,0,0,1


In [66]:
sub_test

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,83.96,3.730,4.830486,35.740000,25.662469,77.866767,1.766674,6.908,1,14.323393,...,0,0,0,0,0,0,0,0,0,0
1,74.97,3.717,15.975015,7.000000,9.465995,28.784088,5.158807,6.908,0,14.232529,...,0,0,0,0,0,0,0,0,0,0
2,69.87,3.721,5.895598,7.640000,2.362720,19.088349,4.167407,6.908,0,14.225052,...,0,0,0,0,0,0,0,0,0,0
3,76.08,3.666,3.402681,7.640000,0.831234,26.796938,3.062140,6.908,0,14.178110,...,0,0,0,0,0,0,0,0,0,0
4,87.65,3.730,9.892535,149.280000,30.715365,55.199849,2.114178,6.565,1,14.456721,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,64.80,3.821,0.353571,39.962783,21.414080,32.316941,1.305095,5.407,0,12.780743,...,0,0,0,0,0,0,0,0,1,0
176,75.70,3.911,10.231840,12.800000,26.513854,35.051876,1.326827,8.684,1,13.549608,...,0,0,0,0,0,0,0,0,0,1
177,67.87,3.948,10.587769,39.962783,2.166247,64.664462,3.400762,8.684,0,13.462029,...,0,0,0,0,0,0,0,0,0,1
178,65.32,4.038,7.844524,92.280000,31.858942,44.909383,5.596207,8.684,0,13.491285,...,0,0,0,0,0,0,0,0,0,1


In [67]:
test

,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Day,...,Store_36,Store_37,Store_38,Store_39,Store_40,Store_41,Store_42,Store_43,Store_44,Store_45
0,68.55,3.617,7.497125,64.672,9.178841,68.363035,2.340449,6.573,0,5,...,0,0,0,0,0,0,0,0,0,0
1,62.99,3.601,1.936193,64.672,4.085642,11.383540,3.825157,6.573,0,12,...,0,0,0,0,0,0,0,0,0,0
2,67.97,3.594,0.882005,64.672,2.483627,1.516583,1.492802,6.573,0,19,...,0,0,0,0,0,0,0,0,0,0
3,69.16,3.506,2.399939,31.750,3.022670,19.978456,0.842303,6.573,0,26,...,0,0,0,0,0,0,0,0,0,0
4,70.27,3.617,5.603672,64.672,5.057935,57.211944,2.487132,6.170,0,5,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,46.97,3.755,0.035871,2.610,0.493703,24.271758,0.295443,5.217,0,26,...,0,0,0,0,0,0,0,0,1,0
176,64.89,3.985,4.683902,64.672,9.481108,42.585845,1.510332,8.667,0,5,...,0,0,0,0,0,0,0,0,0,1
177,54.47,4.000,1.815632,64.672,3.974811,11.326089,2.575647,8.667,0,12,...,0,0,0,0,0,0,0,0,0,1
178,56.47,3.969,1.859940,64.672,1.602015,8.272324,0.992355,8.667,0,19,...,0,0,0,0,0,0,0,0,0,1


----
# II. Modeling

## 1. Divide `sub_train` and `sub_test`

In [68]:
x_train = sub_train.drop(columns=["Weekly_Sales"])
y_train = sub_train["Weekly_Sales"]

x_eval = sub_test.drop(columns=["Weekly_Sales"])
y_eval = sub_test["Weekly_Sales"]

## 2. Choose a suitable model

### a) XGBoost

In [69]:
import time
import xgboost as xgb
from matplotlib import pyplot as plt


start_t = time.time()

model = xgb.XGBRegressor(objective='reg:squarederror', learning_rate=0.1, max_depth = 4, n_estimators = 1000)

model.fit(x_train, y_train, eval_set=[(x_train, y_train),(x_eval, y_eval)], eval_metric='rmse')

pred = np.expm1(model.predict(test))

print(f"[XGBoost] => time: {round(time.time() - start_t, 2)}(sec)")
print(f"{pred[0]}, {pred[1]}, ... , {pred[-1]}")

[0]	validation_0-rmse:11.89747	validation_1-rmse:11.86536
[1]	validation_0-rmse:10.71067	validation_1-rmse:10.67845
[2]	validation_0-rmse:9.64266	validation_1-rmse:9.61041
[3]	validation_0-rmse:8.68158	validation_1-rmse:8.64921
[4]	validation_0-rmse:7.81681	validation_1-rmse:7.77021
[5]	validation_0-rmse:7.03862	validation_1-rmse:6.99182
[6]	validation_0-rmse:6.33850	validation_1-rmse:6.27912
[7]	validation_0-rmse:5.70848	validation_1-rmse:5.64891
[8]	validation_0-rmse:5.14171	validation_1-rmse:5.08191
[9]	validation_0-rmse:4.63195	validation_1-rmse:4.55952
[10]	validation_0-rmse:4.17331	validation_1-rmse:4.10058
[11]	validation_0-rmse:3.76083	validation_1-rmse:3.68792
[12]	validation_0-rmse:3.38994	validation_1-rmse:3.31673
[13]	validation_0-rmse:3.05645	validation_1-rmse:2.98295
[14]	validation_0-rmse:2.75672	validation_1-rmse:2.68292
[15]	validation_0-rmse:2.48735	validation_1-rmse:2.41321
[16]	validation_0-rmse:2.24539	validation_1-rmse:2.17104
[17]	validation_0-rmse:2.02819	valida

C:\Anaconda\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[39]	validation_0-rmse:0.36774	validation_1-rmse:0.30791
[40]	validation_0-rmse:0.35455	validation_1-rmse:0.29742
[41]	validation_0-rmse:0.34295	validation_1-rmse:0.28816
[42]	validation_0-rmse:0.33271	validation_1-rmse:0.28018
[43]	validation_0-rmse:0.32368	validation_1-rmse:0.27396
[44]	validation_0-rmse:0.31545	validation_1-rmse:0.26903
[45]	validation_0-rmse:0.30817	validation_1-rmse:0.26372
[46]	validation_0-rmse:0.30160	validation_1-rmse:0.25939
[47]	validation_0-rmse:0.29568	validation_1-rmse:0.25528
[48]	validation_0-rmse:0.29030	validation_1-rmse:0.25139
[49]	validation_0-rmse:0.28539	validation_1-rmse:0.24859
[50]	validation_0-rmse:0.28081	validation_1-rmse:0.24560
[51]	validation_0-rmse:0.27613	validation_1-rmse:0.24340
[52]	validation_0-rmse:0.27213	validation_1-rmse:0.24073
[53]	validation_0-rmse:0.26838	validation_1-rmse:0.23861
[54]	validation_0-rmse:0.26485	validation_1-rmse:0.23636
[55]	validation_0-rmse:0.26147	validation_1-rmse:0.23405
[56]	validation_0-rmse:0.25823	

[182]	validation_0-rmse:0.09225	validation_1-rmse:0.10215
[183]	validation_0-rmse:0.09142	validation_1-rmse:0.10156
[184]	validation_0-rmse:0.09096	validation_1-rmse:0.10174
[185]	validation_0-rmse:0.09052	validation_1-rmse:0.10140
[186]	validation_0-rmse:0.09009	validation_1-rmse:0.10094
[187]	validation_0-rmse:0.08969	validation_1-rmse:0.10059
[188]	validation_0-rmse:0.08918	validation_1-rmse:0.10057
[189]	validation_0-rmse:0.08881	validation_1-rmse:0.10060
[190]	validation_0-rmse:0.08841	validation_1-rmse:0.10040
[191]	validation_0-rmse:0.08803	validation_1-rmse:0.10004
[192]	validation_0-rmse:0.08717	validation_1-rmse:0.09950
[193]	validation_0-rmse:0.08676	validation_1-rmse:0.09932
[194]	validation_0-rmse:0.08626	validation_1-rmse:0.09895
[195]	validation_0-rmse:0.08591	validation_1-rmse:0.09865
[196]	validation_0-rmse:0.08504	validation_1-rmse:0.09789
[197]	validation_0-rmse:0.08468	validation_1-rmse:0.09729
[198]	validation_0-rmse:0.08427	validation_1-rmse:0.09694
[199]	validati

[324]	validation_0-rmse:0.05491	validation_1-rmse:0.07003
[325]	validation_0-rmse:0.05477	validation_1-rmse:0.06976
[326]	validation_0-rmse:0.05464	validation_1-rmse:0.06974
[327]	validation_0-rmse:0.05455	validation_1-rmse:0.06973
[328]	validation_0-rmse:0.05445	validation_1-rmse:0.06980
[329]	validation_0-rmse:0.05432	validation_1-rmse:0.06933
[330]	validation_0-rmse:0.05424	validation_1-rmse:0.06926
[331]	validation_0-rmse:0.05415	validation_1-rmse:0.06932
[332]	validation_0-rmse:0.05407	validation_1-rmse:0.06929
[333]	validation_0-rmse:0.05399	validation_1-rmse:0.06926
[334]	validation_0-rmse:0.05369	validation_1-rmse:0.06893
[335]	validation_0-rmse:0.05361	validation_1-rmse:0.06876
[336]	validation_0-rmse:0.05353	validation_1-rmse:0.06873
[337]	validation_0-rmse:0.05341	validation_1-rmse:0.06835
[338]	validation_0-rmse:0.05330	validation_1-rmse:0.06832
[339]	validation_0-rmse:0.05323	validation_1-rmse:0.06841
[340]	validation_0-rmse:0.05307	validation_1-rmse:0.06827
[341]	validati

[466]	validation_0-rmse:0.04375	validation_1-rmse:0.05902
[467]	validation_0-rmse:0.04369	validation_1-rmse:0.05870
[468]	validation_0-rmse:0.04366	validation_1-rmse:0.05863
[469]	validation_0-rmse:0.04362	validation_1-rmse:0.05874
[470]	validation_0-rmse:0.04348	validation_1-rmse:0.05876
[471]	validation_0-rmse:0.04340	validation_1-rmse:0.05878
[472]	validation_0-rmse:0.04334	validation_1-rmse:0.05879
[473]	validation_0-rmse:0.04329	validation_1-rmse:0.05862
[474]	validation_0-rmse:0.04322	validation_1-rmse:0.05848
[475]	validation_0-rmse:0.04319	validation_1-rmse:0.05847
[476]	validation_0-rmse:0.04315	validation_1-rmse:0.05842
[477]	validation_0-rmse:0.04313	validation_1-rmse:0.05848
[478]	validation_0-rmse:0.04308	validation_1-rmse:0.05848
[479]	validation_0-rmse:0.04303	validation_1-rmse:0.05838
[480]	validation_0-rmse:0.04297	validation_1-rmse:0.05838
[481]	validation_0-rmse:0.04294	validation_1-rmse:0.05831
[482]	validation_0-rmse:0.04284	validation_1-rmse:0.05811
[483]	validati

[608]	validation_0-rmse:0.03799	validation_1-rmse:0.05408
[609]	validation_0-rmse:0.03794	validation_1-rmse:0.05408
[610]	validation_0-rmse:0.03793	validation_1-rmse:0.05408
[611]	validation_0-rmse:0.03791	validation_1-rmse:0.05406
[612]	validation_0-rmse:0.03784	validation_1-rmse:0.05383
[613]	validation_0-rmse:0.03780	validation_1-rmse:0.05360
[614]	validation_0-rmse:0.03776	validation_1-rmse:0.05365
[615]	validation_0-rmse:0.03774	validation_1-rmse:0.05365
[616]	validation_0-rmse:0.03771	validation_1-rmse:0.05363
[617]	validation_0-rmse:0.03768	validation_1-rmse:0.05364
[618]	validation_0-rmse:0.03765	validation_1-rmse:0.05359
[619]	validation_0-rmse:0.03761	validation_1-rmse:0.05357
[620]	validation_0-rmse:0.03753	validation_1-rmse:0.05360
[621]	validation_0-rmse:0.03748	validation_1-rmse:0.05346
[622]	validation_0-rmse:0.03745	validation_1-rmse:0.05334
[623]	validation_0-rmse:0.03740	validation_1-rmse:0.05335
[624]	validation_0-rmse:0.03737	validation_1-rmse:0.05339
[625]	validati

[750]	validation_0-rmse:0.03359	validation_1-rmse:0.05174
[751]	validation_0-rmse:0.03358	validation_1-rmse:0.05174
[752]	validation_0-rmse:0.03352	validation_1-rmse:0.05176
[753]	validation_0-rmse:0.03350	validation_1-rmse:0.05175
[754]	validation_0-rmse:0.03347	validation_1-rmse:0.05177
[755]	validation_0-rmse:0.03346	validation_1-rmse:0.05178
[756]	validation_0-rmse:0.03344	validation_1-rmse:0.05178
[757]	validation_0-rmse:0.03343	validation_1-rmse:0.05173
[758]	validation_0-rmse:0.03340	validation_1-rmse:0.05163
[759]	validation_0-rmse:0.03336	validation_1-rmse:0.05123
[760]	validation_0-rmse:0.03334	validation_1-rmse:0.05121
[761]	validation_0-rmse:0.03333	validation_1-rmse:0.05121
[762]	validation_0-rmse:0.03328	validation_1-rmse:0.05121
[763]	validation_0-rmse:0.03327	validation_1-rmse:0.05120
[764]	validation_0-rmse:0.03326	validation_1-rmse:0.05118
[765]	validation_0-rmse:0.03322	validation_1-rmse:0.05113
[766]	validation_0-rmse:0.03321	validation_1-rmse:0.05113
[767]	validati

[892]	validation_0-rmse:0.03064	validation_1-rmse:0.05016
[893]	validation_0-rmse:0.03063	validation_1-rmse:0.05016
[894]	validation_0-rmse:0.03060	validation_1-rmse:0.05022
[895]	validation_0-rmse:0.03059	validation_1-rmse:0.05021
[896]	validation_0-rmse:0.03059	validation_1-rmse:0.05021
[897]	validation_0-rmse:0.03057	validation_1-rmse:0.05016
[898]	validation_0-rmse:0.03056	validation_1-rmse:0.05015
[899]	validation_0-rmse:0.03054	validation_1-rmse:0.05014
[900]	validation_0-rmse:0.03054	validation_1-rmse:0.05014
[901]	validation_0-rmse:0.03053	validation_1-rmse:0.05013
[902]	validation_0-rmse:0.03051	validation_1-rmse:0.05011
[903]	validation_0-rmse:0.03051	validation_1-rmse:0.05011
[904]	validation_0-rmse:0.03050	validation_1-rmse:0.05012
[905]	validation_0-rmse:0.03045	validation_1-rmse:0.05013
[906]	validation_0-rmse:0.03042	validation_1-rmse:0.05009
[907]	validation_0-rmse:0.03039	validation_1-rmse:0.05008
[908]	validation_0-rmse:0.03038	validation_1-rmse:0.05011
[909]	validati

----
# III. Submission

In [70]:
"""
import os

def name(integer):
    return str(integer).zfill(2)
    
savetime = dt.datetime.now()
folder = "-".join(map(name, [savetime.year, savetime.month, savetime.day]))
sub_folder = name(savetime.hour) + '：' + name(savetime.minute) + '：' + name(savetime.second)

for model in predictions:
    submission["Weekly_Sales"] = predictions[model]
    os.makedirs(f"dataset/submissions/{folder}", exist_ok=True)
    submission.to_csv(f"dataset/submissions/{folder}/{sub_folder+' ('+model+')'}.csv", index=False)
"""

'\nimport os\n\ndef name(integer):\n    return str(integer).zfill(2)\n    \nsavetime = dt.datetime.now()\nfolder = "-".join(map(name, [savetime.year, savetime.month, savetime.day]))\nsub_folder = name(savetime.hour) + \'：\' + name(savetime.minute) + \'：\' + name(savetime.second)\n\nfor model in predictions:\n    submission["Weekly_Sales"] = predictions[model]\n    os.makedirs(f"dataset/submissions/{folder}", exist_ok=True)\n    submission.to_csv(f"dataset/submissions/{folder}/{sub_folder+\' (\'+model+\')\'}.csv", index=False)\n'